In [1]:
import numpy as np
import pandas as pd
import EOS_extension
import TOVsolver
import MR_generator
import os
import time

In [2]:
EOS_start = pd.read_table('data/EOSCEFTVE1.dat', header=None).to_numpy()

In [3]:
def generate_tables(number_tables, EOS_start, nsamp_EOS, ext_type, MRL_size=100, maxm_thresh=1.8):
    datapath = os.getcwd() + "/data/"
    EOSdir_name = ext_type+str(nsamp_EOS)+'EOS'
    MRLdir_name = ext_type+str(nsamp_EOS)+'MRL'
    
    if not(EOSdir_name in os.listdir(datapath)) and not(MRLdir_name in os.listdir(datapath)):
        os.makedirs(datapath+EOSdir_name)
        os.makedirs(datapath+MRLdir_name)
        
        
    numlist = []
    for file in os.listdir(datapath+EOSdir_name):
        if '.dat' in file:
            numlist.append(int(file[:-4]))
    
    if len(numlist) == 0:
        filenumstart = 0
    else:
        filenumstart = max(numlist) + 1

    i = 0
    done_25 = False
    done_50 = False
    done_75 = False
    num_failed = 0
    while i < number_tables:
        
        if ext_type == 'poly':
            EOS, ns, gammas, Ks = EOS_extension.extend(EOS_start, nsamp=nsamp_EOS,
                                                            ext_type=ext_type, max_gamma=9)
            param_string = "ns =" + str(ns) + ' gammas =' + str(gammas) + ' Ks =' + str(Ks)
        else:
            EOS, ns, cs = EOS_extension.extend(EOS_start, nsamp=nsamp_EOS, ext_type=ext_type)
            param_string = "ns =" + str(ns) + ' cs =' + str(cs)
        
        # extend EOS and store parameters

        MRL_table = TOVsolver.solve(EOS, MRL_size) #solve tov
        raw_mass = MRL_table[:,0]
        raw_radius = MRL_table[:,1]
        raw_Lambda = MRL_table[:,2]
        
        # create boolean arrays to test if points are good
        m2big = raw_mass < 4
        r2small = raw_radius > 7
        # the bool array of points we will keep
        keep = np.logical_and(m2big, r2small)
        # define new arrays we will keep
        radius = raw_radius[keep]
        mass = raw_mass[keep]
        Lambda = raw_Lambda[keep]
        
        #check if maximum mass is realistic
        maxm = np.max(raw_mass)
        if maxm < maxm_thresh:
#             print("maximum mass is too low")
            num_failed += 1
            i -= 1
        else:
            leng = len(radius) # get number of physical points
            MRL = np.zeros((leng, 3)) # initialize MRL table
            MRL[:,0], MRL[:,1], MRL[:,2] = mass, radius, Lambda # put into table

            EOSname = datapath + EOSdir_name + '/' + str(filenumstart + i) + '.dat' # make names for file
            MRLname = datapath + MRLdir_name + '/' + str(filenumstart + i) + '.dat'
            
#             print("saving file...")

            np.savetxt(EOSname, EOS, header=param_string) # save files
            np.savetxt(MRLname, MRL)
    
        
        if i*100/number_tables > 25 and done_25==False:
            print("The run is 25% complete")
            done_25=True
        elif i*100/number_tables > 50 and done_50==False:
            print("The run is 50% complete")
            done_50=True
        elif i*100/number_tables > 75 and done_75==False:
            print("The run is 75% complete")
            done_75=True
        
        i +=1
    print(str(num_failed)+' iterations failed for '+str(number_tables)+' successful tables.')

In [2]:
datapath = os.getcwd() + '/data/'
nsim = 400
ext_type = 'cs'
nsamp = 8

_, MRL = MR_generator.make_MRL(datapath, nsamp, ext_type, save=True, newsavename=None)

saving files...
Generation took 23.122812271118164 seconds to complete.


In [7]:
nsim = 500
nsamp = 8
ext_type = 'cs'

start_time = time.time()
generate_tables(nsim, EOS_start, nsamp, ext_type)
print(time.time()-start_time)

The run is 25% complete
The run is 50% complete
The run is 75% complete
670 iterations failed for 500 successful tables.
32299.29222393036


In [8]:
nsim = 500
nsamp = 8
ext_type = 'cs'
start_time = time.time()
generate_tables(nsim, EOS_start, nsamp, ext_type)
print(time.time()-start_time)

The run is 25% complete
The run is 50% complete
The run is 75% complete
595 iterations failed for 500 successful tables.
26011.178444862366


In [7]:
def get_r1p4(MRL_table):
    max_index = np.argmax(MRL_table[:,0])
    clipped_MRL = MRL_table[:max_index-1,:]
        
    diffMRL = np.ediff1d(clipped_MRL[:,0])
    drop_in_m = np.argmax(diffMRL < 0) # index of first drop in mass
    above_m = np.argmax(clipped_MRL[:,0]>clipped_MRL[drop_in_m,0])
    
    keep = np.zeros(clipped_MRL[:,0].shape[0], dtype=bool)
    for i in range(clipped_MRL[:,0].shape[0]):
        keep[i] = i < drop_in_m or i > above_m
        
    fixed_MRL = clipped_MRL[keep,:]
    r_func = scipy.interpolate.CubicSpline(fixed_MRL[:,0], fixed_MRL[:,1])
    r1p4 = r_func(1.4)
    
    return r1p4

In [10]:
import pandas as pd
import scipy.interpolate

datapath = os.getcwd() + "/data/"

nsamp = 5
ext_type = 'cs'

ntrys = 500 
for j in range(ntrys):
    new_EOS, param_string, new_MRL = MR_generator.make_MRL(datapath, nsamp, ext_type, save=True)
    new_r1p4 = get_r1p4(new_MRL)

    if new_r1p4 > 8 and new_r1p4 < 10.8:
        print("saving to uniform")
        MR_generator.save_set(new_EOS, param_string, new_MRL, datapath=datapath,
                              nsamp=nsamp, ext_type=ext_type, newsavename="uniform")

saving files...
Generation took 113.32095289230347 seconds to complete.
saving files...
Generation took 108.41839790344238 seconds to complete.
saving files...
Generation took 21.133479833602905 seconds to complete.
saving files...
Generation took 21.35276484489441 seconds to complete.
saving files...
Generation took 42.74658012390137 seconds to complete.
saving files...
Generation took 22.856194019317627 seconds to complete.
saving files...
Generation took 156.2005970478058 seconds to complete.
saving files...
Generation took 67.03167796134949 seconds to complete.
saving files...
Generation took 71.37365984916687 seconds to complete.
saving to uniform
saving files...
saving files...
Generation took 153.08268809318542 seconds to complete.
saving files...
Generation took 109.16907811164856 seconds to complete.
saving files...
Generation took 21.573279857635498 seconds to complete.
saving files...
Generation took 44.488763093948364 seconds to complete.
saving files...
Generation took 108

saving files...
Generation took 131.29476499557495 seconds to complete.
saving files...
Generation took 65.62033915519714 seconds to complete.
saving files...
Generation took 66.16705894470215 seconds to complete.
saving files...
Generation took 47.46088409423828 seconds to complete.
saving to uniform
saving files...
saving files...
Generation took 21.367910146713257 seconds to complete.
saving files...
Generation took 199.742604970932 seconds to complete.
saving files...
Generation took 64.58950185775757 seconds to complete.
saving files...
Generation took 47.08457803726196 seconds to complete.
saving files...
Generation took 353.17995595932007 seconds to complete.
saving files...
Generation took 65.0446310043335 seconds to complete.
saving files...
Generation took 21.55877923965454 seconds to complete.
saving files...
Generation took 200.56081986427307 seconds to complete.
saving files...
Generation took 46.19446802139282 seconds to complete.
saving files...
Generation took 153.20789

saving files...
Generation took 44.29162001609802 seconds to complete.
saving files...
Generation took 21.349942207336426 seconds to complete.
saving files...
Generation took 43.596583127975464 seconds to complete.
saving files...
Generation took 22.05846095085144 seconds to complete.
saving files...
Generation took 43.76265001296997 seconds to complete.
saving files...
Generation took 88.4184730052948 seconds to complete.
saving files...
Generation took 46.7621009349823 seconds to complete.
saving files...
Generation took 44.67419195175171 seconds to complete.
saving files...
Generation took 22.383291006088257 seconds to complete.
saving files...
Generation took 110.77948188781738 seconds to complete.
saving files...
Generation took 69.8166401386261 seconds to complete.
saving files...
Generation took 65.32066893577576 seconds to complete.
saving files...
Generation took 154.92430186271667 seconds to complete.
saving files...
Generation took 159.0255482196808 seconds to complete.
savi

KeyboardInterrupt: 